In [1]:
import pygame
import config_goose as config
from goose_functions import (
    load_goose, draw_player, load_bread, draw_bread,
    spawn_breads, spawn_balls, draw_hud, game_over, draw_title_screen, spawn_new_ball
)

def main():
    pygame.init()
    screen = pygame.display.set_mode((config.WIDTH, config.HEIGHT))
    pygame.display.set_caption("not that goose game")
    clock = pygame.time.Clock()

    # load art
    load_goose("R")
    load_bread()

    # player setup
    px, py = config.WIDTH // 2, config.HEIGHT // 2
    facing = "R"

    # game objects
    breads = spawn_breads(config.N_BREAD)
    balls = spawn_balls(config.N_BALLS, config.BALL_SPEEDS)
    score = 0


    # ---- NEW: game state ----
    game_state = "start"   # can be "start", "playing", or "gameover"

    running = True
    while running:
        dt = clock.tick(config.FPS) / 1000.0
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        keys = pygame.key.get_pressed()
        if keys[pygame.K_ESCAPE]:
            running = False
            game_over(screen, won=False, score=score)

        # Wait until user selects start SPACE
        if game_state == "start":
            screen.fill(config.BG_COLOR)
            draw_title_screen(screen)
            pygame.display.flip()

            if keys[pygame.K_SPACE]:
                # begin the actual game
                game_state = "playing"
            continue  # skip the rest until started

        # Game has started! is playing
        if game_state == "playing":
            dx = (keys[pygame.K_d] or keys[pygame.K_RIGHT]) - (keys[pygame.K_a] or keys[pygame.K_LEFT])
            dy = (keys[pygame.K_s] or keys[pygame.K_DOWN])  - (keys[pygame.K_w] or keys[pygame.K_UP])
            if dx and dy:
                inv = 1 / 2 ** 0.5
                dx *= inv
                dy *= inv

            if dx > 0:  facing = "R"
            elif dx < 0: facing = "L"
            if dx or dy:
                if pygame.time.get_ticks() % 300 < 150:
                    load_goose(facing + "S")
                else:
                    load_goose(facing)
            else:
                load_goose(facing)

            px += dx * config.PLAYER_SPEED * dt
            py += dy * config.PLAYER_SPEED * dt
            px = max(16, min(config.WIDTH - 16, px))
            py = max(16, min(config.HEIGHT - 16, py))

            player_rect = pygame.Rect(0, 0, *config.PLAYER_IMAGE_SIZE)
            player_rect.center = (int(px), int(py))

            # move balls
            for b in balls:
                b.move(dt)

            # collision with balls = lose
            for b in balls:
                if player_rect.colliderect(b.rect):
                    game_over(screen, won=False, score=score)
                    running = False
                    break
            if not running:
                break

            # collect bread
            remaining = []
            for br in breads:
                if player_rect.colliderect(br.rect):
                    score += 1
                    balls.append(spawn_new_ball())
                else:
                    remaining.append(br)
            breads = remaining

            if len(breads) == 0:
                game_over(screen, won=True, score=score)
                running = False
                break

            # draw everything
            screen.fill(config.BG_COLOR)
            for br in breads:
                draw_bread(screen, br.center)
            for b in balls:
                b.draw(screen)
            draw_player(screen, px, py)
            draw_hud(screen, score, len(breads))
            pygame.display.flip()

    pygame.quit()



if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
